# WeatherPy

---

## Starter Code to Generate Random Geographic Coordinates and a List of Cities

In [ ]:
%matplotlib widget

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy import stats
from scipy.stats import linregress

# Impor the OpenWeatherMap API key
from api_keys import weather_api_key

# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy

### Generate the Cities List by Using the `citipy` Library

In [ ]:
# Empty list for holding the latitude and longitude combinations
lat_lngs = []

# Empty list for holding the cities names
cities = []

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
print(f"Number of cities in the list: {len(cities)}")

---

## Requirement 1: Create Plots to Showcase the Relationship Between Weather Variables and Latitude

### Use the OpenWeatherMap API to retrieve weather data from the cities list generated in the started code

In [ ]:
# Set the API base URL
url = "http://api.openweathermap.org/data/2.5/weather?"

# Define an empty list to fetch the weather data for each city
city_data = []

# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in our list to fetch weather data
for i, city in enumerate(cities):
        
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 0

    # Create endpoint URL with each city
    city_url = f"{url}appid={weather_api_key}&q={city}&units=metric"
    
    # Log the url, record, and set numbers
    print("Processing Record %s of Set %s | %s" % (record_count, set_count, city))

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()

        # Parse out latitude, longitude, max temp, humidity, cloudiness, wind speed, country, and date
        city_lat = city_weather['coord']['lat']
        city_lng = city_weather['coord']['lon']
        city_max_temp = city_weather['main']['temp_max']
        city_humidity = city_weather['main']['humidity']
        city_clouds = city_weather['clouds']['all']
        city_wind = city_weather['wind']['speed']
        city_country = city_weather['sys']['country']
        city_date = city_weather['dt']

        # Append the City information into city_data list
        city_data.append({"City": city, 
                          "Lat": city_lat, 
                          "Lng": city_lng, 
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass
              
# Indicate that Data Loading is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

In [ ]:
# Convert the cities weather data into a Pandas DataFrame
city_data_df = pd.DataFrame(city_data)

# Show Record Count
city_data_df.count()

In [ ]:
# Display sample data
city_data_df.head()

In [ ]:
# Export the City_Data into a csv
city_data_df.to_csv("../output_data/cities.csv", index_label="City_ID")

In [ ]:
# Read saved data
city_data_df = pd.read_csv("../output_data/cities.csv", index_col="City_ID")

# Display sample data
city_data_df.head()

### Create the Scatter Plots Requested

#### Latitude Vs. Temperature

In [ ]:
# Build scatter plot for latitude vs. temperature
x_val = city_data_df['Lat']
y_val = city_data_df['Max Temp']
plt.scatter(x_val, y_val)

# Incorporate the other graph properties
plt.xlabel('Latitude')
plt.ylabel('Temperature in Celsius')
plt.title("City Latitude vs Temperature")
plt.grid(True)

# Save the figure
plt.savefig("../output_data/Fig1.png")

# Show plot
plt.show()

#### Latitude Vs. Humidity

In [ ]:
plt.clf()

# Build the scatter plots for latitude vs. humidity
x_val = city_data_df['Lat']
y_val = city_data_df['Humidity']
plt.scatter(x_val, y_val)


# Incorporate the other graph properties
plt.xlabel('Latitude')
plt.ylabel('Humidity %')
plt.title("City Latitude vs Humidity")
plt.grid(True)

# Save the figure
plt.savefig("../output_data/Fig2.png")

# Show plot
plt.show()

#### Latitude Vs. Cloudiness

In [ ]:
# Build the scatter plots for latitude vs. cloudiness
plt.clf()

x_val = city_data_df['Lat']
y_val = city_data_df['Cloudiness']
plt.scatter(x_val, y_val)

# Incorporate the other graph properties
plt.xlabel('Latitude')
plt.ylabel('Cloudiness %')
plt.title("City Latitude vs Cloudiness")
plt.grid(True)


# Save the figure
plt.savefig("../output_data/Fig3.png")

# Show plot
plt.show()

#### Latitude vs. Wind Speed Plot

In [ ]:
# Build the scatter plots for latitude vs. wind speed
plt.clf()

x_val = city_data_df['Lat']
y_val = city_data_df['Wind Speed']
plt.scatter(x_val, y_val)

# Incorporate the other graph properties
plt.xlabel('Latitude')
plt.ylabel('Wind Speed (meter/sec)')
plt.title("City Latitude vs Wind Speed")
plt.grid(True)


# Save the figure
plt.savefig("../output_data/Fig4.png")

# Show plot
plt.show()

---

## Requirement 2: Compute Linear Regression for Each Relationship


In [ ]:
# Define a function to create Linear Regression plots
def regress(x_val, y_val, yheader, hemisphere):
    (slope, intercept, rvalue, pvalue, stderr) = stats.linregress(x_val, y_val)
    regress_val = x_val * slope + intercept
    plt.clf()
    plt.scatter(x_val, y_val)
    plt.plot(x_val, regress_val, "r-")
    plt.xlabel('Latitude')
    plt.ylabel(f'{yheader}')
    plt.title(f"City Latitude vs {yheader} ({hemisphere} Hemisphere)")
    plt.grid(True)
    plt.savefig(f"../output_data/regress_{yheader}_{hemisphere}.png")
    plt.show()
    
    return

In [ ]:
# Create a DataFrame with the Northern Hemisphere data (Latitude >= 0)
northern_hemi_df = city_data_df.loc[(city_data_df['Lat']>= 0),:]

# Display sample data
northern_hemi_df.head()

In [ ]:
# Create a DataFrame with the Southern Hemisphere data (Latitude < 0)
southern_hemi_df = city_data_df.loc[(city_data_df['Lat']< 0),:]

# Display sample data
southern_hemi_df.head()

###  Temperature vs. Latitude Linear Regression Plot

In [ ]:
# Linear regression on Northern Hemisphere
regress(northern_hemi_df["Lat"], northern_hemi_df["Max Temp"], "Maximum Temperature in Celsius", "Northern")


In [ ]:
# Linear regression on Southern Hemisphere
regress(southern_hemi_df["Lat"], southern_hemi_df["Max Temp"], "Maximum Temperature in Celsius", "Southern")

**Discussion about the linear relationship:**  
It can be seen from the linear regression lines that maximum temperatures increase as we approach the Equator (latitude = 0).

### Humidity vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
regress(northern_hemi_df["Lat"], northern_hemi_df["Humidity"], "Humidity %", "Northern")

In [ ]:
# Southern Hemisphere
regress(southern_hemi_df["Lat"], southern_hemi_df["Humidity"], "Humidity %", "Southern")

**Discussion about the linear relationship:**  
There is no apparent linear relationship based on the linear regression. However, based on the scatterplots, it appears that the humidity around the Equator and the poles are generally high and the humidities around 20 to 30 and -20 to -30 latitues vary greatly.

### Cloudiness vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
regress(northern_hemi_df["Lat"], northern_hemi_df["Cloudiness"], "Cloudiness Percentage", "Northern")


In [ ]:
# Southern Hemisphere
regress(southern_hemi_df["Lat"], southern_hemi_df["Cloudiness"], "Cloudiness Percentage", "Southern")

**Discussion about the linear relationship:**  
Cloudiness appears to decrease as latitude increases just looking at the lines, however, the scatter plot points are spread out all over the graphs, suggesting that the linear relationship shown is likely not a correlation.

### Wind Speed vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
regress(northern_hemi_df["Lat"], northern_hemi_df["Wind Speed"], "Wind Speed in meters per seconds", "Northern")

In [ ]:
# Southern Hemisphere
regress(southern_hemi_df["Lat"], southern_hemi_df["Wind Speed"], "Wind Speed in Meters per Seconds", "Southern")

**Discussion about the linear relationship:**  
In the northern hemisphere, the wind speeds are generally between 0-5m/s based on the scatter plot and the linear regression shows no obvious correlation between the two. In the southern hemisphere, there is more variation in wind speeds across all the latitudes and the linear regression line is similar (showing no obvious correlation).